In [3]:
pip install opencv-python

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 991.0 kB/s eta 0:00:40
   ---------------------------------------- 0.4/38.8 MB 4.6 MB/s eta 0:00:09
    --------------------------------------- 0.9/38.8 MB 7.2 MB/s eta 0:00:06
   - -------------------------------------- 1.5/38.8 MB 8.5 MB/s eta 0:00:05
   -- ------------------------------------- 2.0/38.8 MB 9.1 MB/s eta 0:00:05
   -- ------------------------------------- 2.5/38.8 MB 9.5 MB/s eta 0:00:04
   --- ------------------------------------ 3.1/38.8 MB 9.8 MB/s eta 0:00:04
   --- ------------------------------------ 3.6/38.8 MB 10.1 MB/s eta 0:00:04
   ---- ----------------------------------- 4.2/38.8 MB 10.3 MB/s eta 0:00:04
   ---- ----------------------------------- 4.8/38.8 MB 10.5 MB/s eta 0:00:04
   ----- ---------------------------------- 5.3/38.8 MB 10.7 MB/s eta 0:00:04
   ------ --------------------------------- 5.9/38.8 MB 10.7 MB/s eta 0:00:04

In [4]:
pip install mediapipe

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached jax-0.4.34-py3-none-any.whl.metadata (22 kB)
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached sounddevice-0.5.0-py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
   ---------------------------------------- 0.0/50.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/50.8 MB 4.0 MB/s eta 0:00:13
   ---------------------------------------- 0.4/50.8 MB 5.4 MB/s eta 0:00:10
    --------------------------------------- 0.9/50.8 MB 7.3 MB/s eta 0:00:07
   - -------------------------------------- 1.4/50.8 MB 8.3 MB/s eta 0:00:06
   - -------------------------------------- 1.8/50.8 MB 8.4 MB/s eta 0:00:06
   - ---------------------------------

In [6]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [32]:
!pip install screen-brightness-control

In [34]:
!pip install pycaw

   ---------------------------------------- 0.0/226.8 kB ? eta -:--:--
   ------- -------------------------------- 41.0/226.8 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------  225.3/226.8 kB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 226.8/226.8 kB 2.3 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade protobuf

In [35]:
import cv2
import numpy as np
import mediapipe as mp
import screen_brightness_control as sbc
from math import hypot
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL


In [36]:
def main():
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))
    volRange = volume.GetVolumeRange()
    minVol, maxVol, _ = volRange

    mpHands = mp.solutions.hands
    hands = mpHands.Hands(
        static_image_mode=False,
        model_complexity=1,
        min_detection_confidence=0.75,
        min_tracking_confidence=0.75,
        max_num_hands=2)

    draw = mp.solutions.drawing_utils
    cap = cv2.VideoCapture(0)
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.flip(frame, 1)
            frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            processed = hands.process(frameRGB)

            left_landmark_list, right_landmark_list = get_left_right_landmarks(frame, processed, draw, mpHands)
            
            if left_landmark_list:
                left_distance = get_distance(frame, left_landmark_list)
                b_level = np.interp(left_distance, [50, 220], [0, 100])
                sbc.set_brightness(int(b_level))

            
            if right_landmark_list:
                right_distance = get_distance(frame, right_landmark_list)
                vol = np.interp(right_distance, [50, 220], [minVol, maxVol])
                volume.SetMasterVolumeLevel(vol, None)

            cv2.imshow('Image', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

In [37]:
def get_left_right_landmarks(frame, processed, draw, mpHands):
    left_landmark_list = []
    right_landmark_list = []

    if processed.multi_hand_landmarks:
        for handlm in processed.multi_hand_landmarks:
            for idx, found_landmark in enumerate(handlm.landmark):
                height, width, _ = frame.shape
                x, y = int(found_landmark.x * width), int(found_landmark.y * height)
                if idx == 4 or idx == 8:
                    landmark = [idx, x, y]
                    if handlm == processed.multi_hand_landmarks[0]:
                        left_landmark_list.append(landmark)
                    elif handlm == processed.multi_hand_landmarks[1]:
                        right_landmark_list.append(landmark)

            draw.draw_landmarks(frame, handlm, mpHands.HAND_CONNECTIONS)

    return left_landmark_list, right_landmark_list

In [38]:
def get_distance(frame, landmark_ist):
    if len(landmark_ist) < 2:
        return
    (x1, y1), (x2, y2) = (landmark_ist[0][1], landmark_ist[0][2]), \
        (landmark_ist[1][1], landmark_ist[1][2])
    cv2.circle(frame, (x1, y1), 7, (0, 255, 0), cv2.FILLED)
    cv2.circle(frame, (x2, y2), 7, (0, 255, 0), cv2.FILLED)
    cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
    L = hypot(x2 - x1, y2 - y1)

    return L

In [ ]:
if __name__ == '__main__':
  main()

C:\Users\Goutham\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
